![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-13b-instruct-v2` to analyze car rental customer satisfaction from text

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of text sentiment analysis in watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to use `ibm/granite-13b-instruct-v2` model to analyze customer satisfaction from text.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


### Install and import the `datasets` and dependecies

In [1]:
!pip install wget | tail -n 1
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [7]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Enter your api key: ")
}

Enter your api key: ··········


### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [8]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Please enter your project_id (hit enter): adbd9616-e904-4d86-a77a-676452bbe4d7


<a id="data"></a>
## Data loading

Download the `car_rental_training_data` dataset. The dataset provides insight about customers opinions on car rental. It has a label that consists of values: unsatisfied, satisfied.

In [9]:
import wget
import pandas as pd

filename = 'car_rental_training_data.csv'
url = 'https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/cars-4-you/car_rental_training_data.csv'

if not os.path.isfile(filename):
    wget.download(url, out=filename)

df = pd.read_csv("car_rental_training_data.csv", sep=';')
data = df[['Customer_Service', 'Satisfaction']]

Examine downloaded data.

In [10]:
data.head()

Customer_Service  Satisfaction
0  I thought the representative handled the initi...             0
1  I have had a few recent rentals that have take...             0
2  car cost more because I didn't pay when I rese...             0
3  I didn't get the car I was told would be avail...             0
4  If there was not a desired vehicle available t...             1

Prepare train and test sets.

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
comments = list(test.Customer_Service)
satisfaction = list(test.Satisfaction)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to <a href="https://ibm.github.io/watson-machine-learning-sdk/model.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes" target="_blank" rel="noopener no referrer">documentation</a>.

In [12]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


You need to specify `model_id` that will be used for inferencing:

In [13]:
model_id = ModelTypes.GRANITE_13B_INSTRUCT_V2

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watson-machine-learning-sdk/model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [14]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.MAX_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [15]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

### Model's details

In [16]:
model.get_details()

{'model_id': 'ibm/granite-13b-instruct-v2',
 'label': 'granite-13b-instruct-v2',
 'provider': 'IBM',
 'source': 'IBM',
 'functions': [{'id': 'prompt_tune_inferable'},
  {'id': 'prompt_tune_trainable'},
  {'id': 'text_generation'}],
 'short_description': 'The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.',
 'long_description': 'Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'input_tier': 'class_1',
 'output_tier': 'class_1',
 'number_params': '13b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction',
  'translation'],
 'tasks': [{'id': 'question_answering',
   'ratings': {'quality': 3},

<a id="predict"></a>
## Analyze the satisfaction

#### Prepare prompt and generate text

In [17]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no"""

In [20]:
prompt1 = "\n".join([instruction, "Comment:" + comments[4], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:We had an issue with our last vacation car rental.  Instead of crediting the rental amount, the company issued a certificate for one free day (could have been 2, I don't exactly remember).  I would have rather had the credit for our current rental.
Satisfied:


Analyze the sentiment for a sample of zero-shot input from the test set.

In [21]:
print(model.generate_text(prompt=prompt1))

no


### Calculate the accuracy

In [22]:
sample_size = 2
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:2]]
results = model.generate_text(prompt=prompts_batch)

In [23]:
print(prompts_batch[0])

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:The car was broken. They couldn't find a replacement. I've waster over 2 hours.
Satisfied:no
Comment:I expected customer service to recognize that I rent from their company every single day. I expect some difference for this loyalty.
Satisfied:


In [24]:
from sklearn.metrics import accuracy_score

label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 0.5


In [25]:
print('true', y_true, '\npred', results)

true ['no', 'yes'] 
pred ['no', 'no']


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.

 You learned how to analyze car rental customer satisfaction with watsonx.ai foundation model.

 Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts.

### Authors

**Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.